<a href="https://colab.research.google.com/github/kuds/rl-drone/blob/main/RL%20Drone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mujoco

# Set up GPU rendering.
from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# Check if installation was succesful.
try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# Other imports and helper functions
import time
import itertools
import numpy as np

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()

In [2]:
!pip install robot_descriptions
!pip install gymnasium
!pip install stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.8/99.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
print(f"Python Version: {platform.python_version()}")
print(f"Torch Version: {version('torch')}")
print(f"Is Cuda Available: {torch.cuda.is_available()}")
print(f"Cuda Version: {torch.version.cuda}")
print(f"Gymnasium Version: {version('gymnasium')}")
print(f"Numpy Version: {version('numpy')}")
print(f"Mujoco Version: {version('mujoco')}")
print(f"Stable-Baselines3 Version: {version('stable-baselines3')}")
print(f"Matplotlib Version: {version('matplotlib')}")

Python Version: 3.11.13
Torch Version: 2.6.0+cu124
Is Cuda Available: True
Cuda Version: 12.4
Gymnasium Version: 1.2.0
Numpy Version: 2.0.2
Mujoco Version: 3.3.4
Stable-Baselines3 Version: 2.7.0
Matplotlib Version: 3.10.0


In [5]:
rl_type = "SAC"
env_str = "BitCrazy"
log_dir = "./logs/{}".format(env_str)
name_prefix = "bit_crazy"

In [6]:
hyperparams = {
    "env_str": env_str,
    "rl_type": rl_type,
    "eval_freq": 25_000,
    "n_envs": 4,
    "min_force": 100.0,
    "total_timesteps": 1_500_000,
    "log_dir": log_dir,
    "episode_length": 1_000
}

In [7]:
csv_header = ["ball_velocity_x",
              "ball_velocity_y",
              "ball_velocity_z",
              "ball_accelerometer_x",
              "ball_accelerometer_y",
              "ball_accelerometer_z",
              "from_to_x1",
              "from_to_y1",
              "from_to_z1",
              "from_to_x2",
              "from_to_y2",
              "from_to_z2",
              "touch_sensor",
              "reward",
              "total_reward",
              "done"]

In [7]:
class VideoRecordCallback(BaseCallback):
    def __init__(
        self,
        save_path: str,
        video_length: int,
        save_freq: int = 5_000,
        name_prefix: str ="rl_model",
        verbose: int = 0):

        super().__init__(verbose)
        self.save_freq = save_freq
        self.video_length = video_length
        self.save_path = save_path
        self.name_prefix = name_prefix
        # Those variables will be accessible in the callback
        # (they are defined in the base class)
        # The RL model
        # self.model = None  # type: BaseAlgorithm
        # An alias for self.model.get_env(), the environment used for training
        # self.training_env # type: VecEnv
        # Number of time the callback was called
        # self.n_calls = 0  # type: int
        # num_timesteps = n_envs * n times env.step() was called
        # self.num_timesteps = 0  # type: int
        # local and global variables
        # self.locals = {}  # type: Dict[str, Any]
        # self.globals = {}  # type: Dict[str, Any]
        # The logger object, used to report things in the terminal
        # self.logger # type: stable_baselines3.common.logger.Logger
        # Sometimes, for event callback, it is useful
        # to have access to the parent object
        # self.parent = None  # type: Optional[BaseCallback]

    def _on_step(self) -> bool:
        if self.n_calls % self.save_freq == 0:

          name_prefix = f"{self.name_prefix}_{self.num_timesteps}"

          # Record video of the best model playing
          rec_val = make_vec_env(make_env, n_envs=1)
          rec_val = VecVideoRecorder(rec_val,
                                    self.save_path,
                                    video_length=self.video_length,
                                    record_video_trigger=lambda x: x == 0,
                                    name_prefix=name_prefix)

          obs = rec_val.reset()
          session_length = 0
          total_reward = 0.0
          csv_file_name = os.path.join(self.save_path, f"{name_prefix}.csv")
          with open(csv_file_name, 'w') as csvfile:
            csv_writer = csv.writer(csvfile, delimiter=',')
            csv_writer.writerow(csv_header)
            for _ in range(self.video_length):
              session_length += 1
              action, _states = self.model.predict(obs)
              obs, rewards, dones, info = rec_val.step(action)
              total_reward += rewards
              #print(info)
              row_data = numpy.concatenate([info[0]["ball_velocity"],
                                            info[0]["ball_accelerometer"],
                                            info[0]["ball_to_paddle"],
                                            [info[0]["touch_sensor"],
                                             rewards[0],
                                             total_reward[0],
                                             dones[0]]])
              row_data = numpy.round(row_data, decimals=4)
              csv_writer.writerow(row_data)
              rec_val.render()

              if dones:
                break

          print(f"Step: {self.num_timesteps} | Session Length: {session_length} |Total Bounces: {int(total_reward[0])}")

          rec_val.close()
        return True


In [10]:
env = BallBounceEnv(render_mode="rgb_array")
print("Observation Space Size: ", env.observation_space.shape)
print('Actions Space: ', env.action_space)
env.close()

OSError: File /content/bit_crazy.xml does not exist

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import mujoco
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

drone_xml = """
<mujoco model="drone">
  <compiler angle="degree" coordinate="local" inertiafromgeom="true"/>
  <option integrator="RK4" timestep="0.01"/>
  <worldbody>
    <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
    <body name="drone" pos="0 0 1">
      <camera name="track" mode="trackcom" pos="0 -3 1" xyaxes="1 0 0 0 0 1"/>
      <geom type="box" size=".1 .1 .02" rgba=".9 .9 .9 1" mass="1"/>
      <joint type="free" limited="false"/>
      <!-- Rotors -->
      <body name="rotor1_body" pos="0.1 0.1 0">
        <joint name="rotor1" type="hinge" axis="0 0 1"/>
        <geom type="cylinder" size="0.05 0.01" rgba="0 1 0 1"/>
      </body>
      <body name="rotor2_body" pos="-0.1 0.1 0">
        <joint name="rotor2" type="hinge" axis="0 0 -1"/>
        <geom type="cylinder" size="0.05 0.01" rgba="1 0 0 1"/>
      </body>
      <body name="rotor3_body" pos="0.1 -0.1 0">
        <joint name="rotor3" type="hinge" axis="0 0 -1"/>
        <geom type="cylinder" size="0.05 0.01" rgba="1 0 0 1"/>
      </body>
      <body name="rotor4_body" pos="-0.1 -0.1 0">
        <joint name="rotor4" type="hinge" axis="0 0 1"/>
        <geom type="cylinder" size="0.05 0.01" rgba="0 1 0 1"/>
      </body>
    </body>
    <!-- Target -->
    <body name="target" pos="1 1 2">
        <geom type="sphere" size="0.1" rgba="1 0 0 0.5" />
    </body>
  </worldbody>
  <actuator>
    <motor joint="rotor1" gear="0.1" ctrllimited="true" ctrlrange="0 1000"/>
    <motor joint="rotor2" gear="0.1" ctrllimited="true" ctrlrange="0 1000"/>
    <motor joint="rotor3" gear="0.1" ctrllimited="true" ctrlrange="0 1000"/>
    <motor joint="rotor4" gear="0.1" ctrllimited="true" ctrlrange="0 1000"/>
  </actuator>
</mujoco>
"""

class DroneEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self):
        super(DroneEnv, self).__init__()

        # Load the drone model
        self.model = mujoco.MjModel.from_xml_string(drone_xml)
        self.data = mujoco.MjData(self.model)

        # Define action and observation space
        # Actions are the 4 rotor thrusts
        self.action_space = spaces.Box(low=0, high=1, shape=(4,), dtype=np.float32)

        # Observations: drone_pos(3), drone_vel(3), drone_orient(4), target_pos(3)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(13,), dtype=np.float32)

        # Target position
        self.target_position = np.array([1.0, 1.0, 2.0])

    def step(self, action):
        # Apply action
        self.data.ctrl[:] = action * 1000  # Scale action to actuator range
        mujoco.mj_step(self.model, self.data)

        # Get observation
        obs = self._get_obs()

        # Calculate reward
        drone_pos = self.data.qpos[0:3]
        dist_to_target = np.linalg.norm(drone_pos - self.target_position)

        # Reward for being close to the target
        reward = -dist_to_target

        # Penalty for being tilted
        orientation = self.data.qpos[3:7]
        # A simple measure of tilt: z-component of the drone's up vector
        # For a quaternion [w, x, y, z], the z-axis of the rotated frame is
        # [2*(xz + wy), 2*(yz - wx), 1 - 2*(x^2 + y^2)]
        up_z = 1 - 2 * (orientation[1]**2 + orientation[2]**2)
        reward -= (1 - up_z) * 0.5

        # Penalty for control effort
        reward -= np.sum(action) * 0.01

        # Check if done
        done = False
        if dist_to_target < 0.2:
            reward += 100  # Bonus for reaching the target
            done = True
        if drone_pos[2] < 0.1: # Crashed
            reward -= 100
            done = True

        return obs, reward, done, False, {}

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        # Reset the simulation
        mujoco.mj_resetData(self.model, self.data)

        # Randomize target position
        self.target_position = np.random.rand(3) * 2 + np.array([-1, -1, 1])
        self.model.body('target').pos[:] = self.target_position

        return self._get_obs(), {}

    def _get_obs(self):
        return np.concatenate([
            self.data.qpos[0:3],    # Drone position
            self.data.qvel[0:3],    # Drone linear velocity
            self.data.qpos[3:7],    # Drone orientation (quaternion)
            self.target_position
        ])

    def render(self, mode='human'):
        # This is a placeholder for rendering.
        # For a full visualization, you would need to integrate a renderer
        # like mujoco.viewer.
        pass

# Create a vectorized environment
env = DummyVecEnv([lambda: DroneEnv()])

# Instantiate the PPO model
model = PPO("MlpPolicy", env, verbose=1)

# Train the model
training_timesteps = 10000
model.learn(total_timesteps=training_timesteps)

# Save the model
model.save("ppo_drone")

print("\nTraining finished and model saved as 'ppo_drone.zip'")

# Load the trained model
model = PPO.load("ppo_drone")

# Create a new environment for evaluation
eval_env = DroneEnv()
obs, _ = eval_env.reset()

print("\nStarting evaluation...")
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, _, info = eval_env.step(action)
    if dones:
        print(f"Episode finished after {i+1} timesteps.")
        obs, _ = eval_env.reset()

print("Evaluation finished.")

In [15]:
import mujoco

# Loading a specific model description as an imported module.
from robot_descriptions import cf2_mj_description
model = mujoco.MjModel.from_xml_path(cf2_mj_description.MJCF_PATH)
print(cf2_mj_description.MJCF_PATH)

/root/.cache/robot_descriptions/mujoco_menagerie/bitcraze_crazyflie_2/cf2.xml


In [ ]:
import mujoco
import robot_descriptions.cf2_mj_description as cf2_mj_description
import xml.etree.ElementTree as ET

# Read the XML content from the file path
tree = ET.parse(cf2_mj_description.MJCF_PATH)
root = tree.getroot()

# Find the worldbody element
worldbody = root.find('worldbody')

# Define the sphere element as a string
sphere_xml = """
<body name="sphere_body" pos="0 0 0.5">
    <geom type="sphere" size="0.1" rgba="0 0 1 1"/>
</body>
"""

# Parse the sphere XML string
sphere_element = ET.fromstring(sphere_xml)

# Append the sphere element to the worldbody
worldbody.append(sphere_element)

# Convert the modified XML back to a string
modified_xml_string = ET.tostring(root, encoding='unicode')

print(modified_xml_string)

# Load the modified model, providing the asset directory
# Note: robot_descriptions.cf2_mj_description does not have ASSETS_PATH.
# You would need to find the correct way to access the asset path for this model.
# For now, I will load without asset_dir, which might cause issues if the model
# relies on external assets.
modified_model = mujoco.MjModel.from_xml_string(modified_xml_string)

# You can now use modified_model for simulation or visualization
print("Sphere added to the model.")

# Example: create data for the modified model
data = mujoco.MjData(modified_model)

# Example: print body names to verify the sphere body is added
for i in range(modified_model.nbody):
    print(f"Body ID: {i}, Name: {modified_model.body(i).name}")

In [8]:
!cat /root/.cache/robot_descriptions/mujoco_menagerie/bitcraze_crazyflie_2/cf2.xml

<?xml version='1.0' encoding='utf-8'?>
<mujoco model="cf2">
  <option integrator="RK4" density="1.225" viscosity="1.8e-5" />
  <compiler inertiafromgeom="false" meshdir="assets" autolimits="true" />
  <default>
    <default class="cf2">
      <default class="visual">
        <geom group="2" type="mesh" contype="0" conaffinity="0" />
      </default>
      <default class="collision">
        <geom group="3" type="mesh" />
      </default>
      <site group="5" />
    </default>
  </default>
  <asset>
    <material name="polished_plastic" rgba="0.631 0.659 0.678 1" />
    <material name="polished_gold" rgba="0.969 0.878 0.6 1" />
    <material name="medium_gloss_plastic" rgba="0.109 0.184 0.0 1" />
    <material name="propeller_plastic" rgba="0.792 0.820 0.933 1" />
    <material name="white" rgba="1 1 1 1" />
    <material name="body_frame_plastic" rgba="0.102 0.102 0.102 1" />
    <material name="burnished_chrome" rgba="0.898 0.898 0.898 1" />
    <mesh file="cf2_0.obj" />
    <mesh fi

In [11]:
f2_mj_description.MJCF_PATH

NameError: name 'f2_mj_description' is not defined